In [3]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

import pandas as pd
import numpy as np
from matplotlib import pyplot
import scipy.misc 
from math import sqrt 
import itertools
from IPython.display import display
%matplotlib inline

# input_data_length = int(input('input_data_length : '))
input_data_length = 60
model_num = 132
num_classes = 2

gdrive_path = '/content/gdrive/My Drive/Colab Notebooks/Project_Stock/'

Made_X = np.load(gdrive_path + 'Made_X/Made_X %s_%s.npy' % (input_data_length, model_num))
Made_Y = np.load(gdrive_path + 'Made_X/Made_Y %s_%s.npy' % (input_data_length, model_num)).reshape(-1, 1)


#       dataset 분리      #
# dataX 구성 : VOLUME, MA, CMO, OBV, RSI, MACD, MACD_SIGNAL, MACD_OSC 
# dataX 구성 : VOLUME, EMA1, EMA2, CMO, OBV, RSI, MACD, MACD_SIGNAL, MACD_OSC 
# Made_X = Made_X[:, :, [-4]]
print(Made_X.shape)
print(Made_Y.shape)
# break

row = Made_X.shape[1]
col = Made_X.shape[2]

from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(Made_X, Made_Y, test_size=0.3,
                                                   shuffle=False)

X_train = X_train.astype('float32')#.reshape(-1, input_data_length, col, 1)
X_val = X_val.astype('float32')#.reshape(-1, input_data_length, col, 1)
print(X_train.shape)
print(X_val.shape)

from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator 

# Data Class Weight
from sklearn.utils import class_weight

print(Y_train[:, 0])
class_weights = class_weight.compute_class_weight('balanced', 
                                                  np.unique(Y_train[:, 0]),
                                                  Y_train[:, 0])
class_weights = dict(enumerate(class_weights))
print(class_weights)
# quit()

Y_train = Y_train.astype('float32')
Y_val = Y_val.astype('float32')
Y_train = np_utils.to_categorical(Y_train, num_classes)
Y_val = np_utils.to_categorical(Y_val, num_classes)
print(Y_train.shape)
print(Y_val.shape)

# datagen = ImageDataGenerator( 
# #     rotation_range = 60,
# #     zoom_range = 0.6,
# #     shear_range = 0.6,
# #     horizontal_flip = True,
# #     width_shift_range=0.6,
# #     height_shift_range=0.6,
#     fill_mode = 'nearest'
#     )

# testgen = ImageDataGenerator( 
#     )
# datagen.fit(X_train)
batch_size = 16

# for X_batch, _ in datagen.flow(X_train, Y_train, batch_size=9):
#     for i in range(0, 9): 
#         pyplot.axis('off') 
#         pyplot.subplot(330 + 1 + i) 
#         pyplot.imshow(X_batch[i].reshape(input_data_length, col))
#     pyplot.axis('off') 
#     pyplot.show() 
#     break
    
    
# train_flow = datagen.flow(X_train, Y_train, batch_size=batch_size) 
# val_flow = testgen.flow(X_val, Y_val, batch_size=batch_size) 


from keras.utils import plot_model
import keras.backend as K
from keras.models import Model, Sequential
import keras.layers as layers
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2
from sklearn.metrics import confusion_matrix

def FER_Model(input_shape=(row, col)):
    # first input model
    visible = layers.Input(shape=input_shape, name='input')
    
    #             LSTM Version            #
    net = layers.LSTM(50, return_sequences=True)(visible)
    net = layers.Dropout(0.2)

    net = layers.LSTM(50, return_sequences=True)(visible)
    net = layers.Dropout(0.2)

    net = layers.LSTM(50, return_sequences=True)(visible)
    net = layers.Dropout(0.2)

    net = layers.LSTM(50, return_sequences=True)(visible)
    net = layers.Dropout(0.2)

    # net = layers.Conv2D(64, kernel_size=3, padding='same')(visible)
    # # net = layers.Activation('relu')(net)
    # net = layers.LeakyReLU()(net)
    # net = layers.MaxPool2D(pool_size=2)(net)

    # shortcut_1 = net

    # net = layers.Conv2D(128, kernel_size=3, padding='same')(net)
    # # net = layers.Activation('relu')(net)
    # net = layers.LeakyReLU()(net)
    # # net = layers.MaxPool2D(pool_size=2)(net)

    # shortcut_2 = net

#     net = layers.Conv2D(256, kernel_size=3, padding='same')(net)
#     # net = layers.Activation('relu')(net)
#     net = layers.LeakyReLU()(net)
#     net = layers.MaxPool2D(pool_size=2)(net)

#     shortcut_3 = net

#     net = layers.Conv2D(128, kernel_size=1, padding='same')(net)
#     # net = layers.Activation('relu')(net)
#     net = layers.LeakyReLU()(net)
#     net = layers.MaxPool2D(pool_size=2)(net)

    # net = layers.Flatten()(net)
    # net = layers.Dense(64)(net)
    # net = layers.LeakyReLU()(net)
    net = layers.Dense(num_classes, activation='softmax')(net)

    # create model 
    model = Model(inputs =visible, outputs = net)
    # summary layers
    print(model.summary())
    
    return model

model = FER_Model()
# from keras.models import load_model
# model = load_model(gdrive_path + 'model/rapid_ascending %s_%s_rnn_macdonly.hdf5' % (input_data_length, model_num))
opt = Adam(lr=0.0001, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
  
    
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
filepath = gdrive_path + "model/rapid_ascending %s_%s_rnn_macdonly.hdf5" % (input_data_length, model_num)
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
checkpoint2 = TensorBoard(log_dir='Tensorboard_graph',
                          histogram_freq=0,
                          write_graph=True,
                          write_images=True)
checkpoint3 = EarlyStopping(monitor='val_acc', patience=200)
callbacks_list = [checkpoint, checkpoint2, checkpoint3]

# keras.callbacks.Callback 로 부터 log 를 받아와 history log 를 작성할 수 있다.

# we iterate 200 times over the entire training set
num_epochs = 1000
history = model.fit(X_train, Y_train,
                    steps_per_epoch=int(len(X_train) / batch_size), 
                    epochs=num_epochs,  
                    verbose=2,  
                    callbacks=callbacks_list,
                    class_weight=class_weights,
                    validation_data=(X_val, Y_val),  
                    validation_steps=int(len(X_val) / batch_size),
                    shuffle=False)



(1409, 60, 14)
(1409, 1)


SyntaxError: ignored